# Hybrid Keras + PyTorch Image Classifier Notebook

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = preprocess_input(x_train), preprocess_input(x_test)

base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(32, 32, 3))
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

features_train = feature_extractor.predict(x_train)
features_test = feature_extractor.predict(x_test)

X_train_torch = torch.tensor(features_train, dtype=torch.float32)
X_test_torch = torch.tensor(features_test, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.long).squeeze()
y_test_torch = torch.tensor(y_test, dtype=torch.long).squeeze()

class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)

model = Classifier(input_dim=2048, num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train_torch)
    loss = criterion(outputs, y_train_torch)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

with torch.no_grad():
    test_outputs = model(X_test_torch)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test_torch).sum().item() / len(y_test_torch)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')